In [2]:
def actualizar_id(filename):

    try:
        with open(filename, 'r') as file:
            lines = file.readlines()
            if lines:
                last_id = int(lines[-1].strip())
            else:
                last_id = 0

        new_id = last_id + 1

        with open(filename, 'a') as file:
            file.write(str(new_id) + '\n')

        return new_id
    
    except FileNotFoundError:
        with open(filename, 'w') as file:
            file.write('1\n')
            return 1

In [8]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

prototxt = "models/deploy.prototxt"
model = "models/res10_300x300_ssd_iter_140000.caffemodel"

net = cv2.dnn.readNetFromCaffe(prototxt, model)

file_id = 'id_list.txt'
current_id = actualizar_id(file_id)
print(current_id)

count = 0
limit = 500

while True:

    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)

    h, w, _ = frame.shape

    frame_resize = cv2.resize(frame, (300, 300))

    blob = cv2.dnn.blobFromImage(frame_resize, 1.0, (300,300), (104, 117, 123))

    net.setInput(blob)
    detections = net.forward()

    box = np.array([0,0,0,0], dtype=np.float64)
    area = 0

    for detection in detections[0][0]:
        if detection[2] > 0.6:
            box_ = detection[3:7] * [w, h, w, h]

            if ((box_[2] - box_[0]) * (box_[3] - box_[1])) > area:
                area = (box_[2] - box_[0]) * (box_[3] - box_[1])
                box = box_

    if area > 0 and len(detections) > 0:
        count += 1

        cv2.imwrite(f'data/User_{current_id}_{count}.jpg', 
                    frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])])
        
        cv2.imshow("ROI", frame[int(box[1]):int(box[3]), int(box[0]):int(box[2])])

        cv2.rectangle(frame, 
                      (int(box[0]), int(box[1])), 
                      (int(box[2]), int(box[3])), 
                      (255,0,0), 2)

    
    k = cv2.waitKey(10)

    if k == ord('q') or count >= limit:
        break

    cv2.imshow("Frame", frame)

cap.release()
cv2.destroyAllWindows()

2
